<a href="https://colab.research.google.com/github/awaiskhan005/DATA-SCIENCE-AND-AI-/blob/main/Franklinn_XGboast_project_PLAN_B.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Warning control
import warnings
warnings.filterwarnings('ignore')
# Import Required Libraries
import os
import joblib
import pandas as pd
import numpy as np
from xgboost import XGBClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report

# Install the 'ta' library if it's not already installed
!pip install ta

# Install the 'python-binance' library if it's not already installed
!pip install python-binance # This line is added to install the missing library

# Now import from 'ta'
from ta.momentum import RSIIndicator
from ta.trend import MACD
from ta.volatility import BollingerBands
from ta.trend import EMAIndicator
from binance.client import Client # This import should now work
from datetime import datetime, timedelta, timezone
import time
import os
import joblib
import pandas as pd
import numpy as np
from xgboost import XGBClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from ta.momentum import RSIIndicator
from ta.trend import MACD
from ta.volatility import BollingerBands
from ta.trend import EMAIndicator
from binance.client import Client
from datetime import datetime, timedelta, timezone
import time

In [ ]:
# Import necessary libraries
import os
import joblib
import pandas as pd
import numpy as np
from xgboost import XGBClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report

# Fetch 6-hour candle data
def fetch_candles(symbol, interval='6h', lookback='500 days'):
    # Example stub for fetching candle data; replace with actual implementation
    data = pd.DataFrame({
        'timestamp': pd.date_range(start='1/1/2020', periods=2000, freq='6H'),
        'open': np.random.random(2000) * 100,
        'high': np.random.random(2000) * 100,
        'low': np.random.random(2000) * 100,
        'close': np.random.random(2000) * 100,
        'volume': np.random.random(2000) * 1000
    })
    data.set_index('timestamp', inplace=True)
    return data

# Calculate indicators
def calculate_indicators(df):
    df['price_return'] = df['close'].pct_change(periods=80)  # Predict 20 days (80 six-hour candles)
    df['rolling_mean'] = df['close'].rolling(window=10).mean()
    df['rolling_std'] = df['close'].rolling(window=10).std()
    df['volume_change'] = df['volume'].pct_change()
    df.dropna(inplace=True)
    return df

# Train and save models
def train_and_save_models(symbols):
    for symbol in symbols:
        print(f"Training model for {symbol}...")
        df = fetch_candles(symbol)
        df = calculate_indicators(df)

        # Define features and target
        X = df[['rolling_mean', 'rolling_std', 'volume_change']]
        y = (df['price_return'] > 0).astype(int)  # Binary classification: 1 if return > 0, else 0

        X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

        model = XGBClassifier()
        model.fit(X_train, y_train)

        # Evaluate and visualize
        probabilities = model.predict_proba(X_test)[:, 1]
        percentage_chances = probabilities * 100
        high_prob_indices = percentage_chances > 60

        print(f"Instances with >60% probability for {symbol}:")
        print(X_test[high_prob_indices])

        # Save model
        model_path = f"{symbol}_xgboost_model.pkl"
        joblib.dump(model, model_path)
        print(f"Model for {symbol} saved at {model_path}")

# Use saved models for prediction
def use_saved_models(symbols):
    for symbol in symbols:
        model_path = f"{symbol}_xgboost_model.pkl"
        if not os.path.exists(model_path):
            print(f"Model for {symbol} not found.")
            continue

        print(f"Loading model for {symbol}...")
        model = joblib.load(model_path)

        # Fetch and preprocess data
        df = fetch_candles(symbol)
        df = calculate_indicators(df)
        X = df[['rolling_mean', 'rolling_std', 'volume_change']]

        # Predict probabilities
        probabilities = model.predict_proba(X)[:, 1]
        percentage_chances = probabilities * 100

        print(f"Probability percentages for {symbol}: {percentage_chances}")

# Integrate predictions into order execution
def execute_buy_order(symbol, probability, threshold=60):
    if probability > threshold:
        print(f"Buy order triggered for {symbol} with probability: {probability:.2f}%")
        # Integration with trading API would go here
    else:
        print(f"No buy signal for {symbol}, probability: {probability:.2f}%")

# Main workflow
def main():
    symbols = ['BTCUSDT', 'ETHUSDT', 'BNBUSDT', 'SOLUSDT']

    # Step 1: Train and save models
    train_and_save_models(symbols)

    # Step 2: Use saved models for prediction
    use_saved_models(symbols)

    # Step 3: Integrate with order execution
    for symbol in symbols:
        model_path = f"{symbol}_xgboost_model.pkl"
        if not os.path.exists(model_path):
            continue

        model = joblib.load(model_path)
        df = fetch_candles(symbol)
        df = calculate_indicators(df)
        X = df[['rolling_mean', 'rolling_std', 'volume_change']]
        probabilities = model.predict_proba(X)[:, 1]

        for prob in probabilities:
            execute_buy_order(symbol, prob)

if __name__ == "__main__":
    main()